<a href="https://colab.research.google.com/github/7jeon8gi/BIZ-AI-LAB/blob/master/Cosine_Similarity__test_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import time
import itertools
import random
import torch
import numpy as np
import pandas as pd
from torch import nn

import os

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
bert_base , vocabulary = get_pytorch_kobert_model()
tokenizer = get_tokenizer()

using cached model
using cached model
using cached model


In [0]:
os.chdir(r"C:\Users\user\Desktop\CYK")
exp_data = pd.read_csv("experiment_data2.csv", encoding='cp949')
os.chdir(r"C:\Users\user\Desktop")
test_data = pd.read_csv("버트테스트문장all.csv", encoding='cp949')

In [0]:
exp_data.head(8)

,tag,type,sentence
0,매매결정,대표문장,"LG유플러스 14,500원에 보유중인데 매도해야 할까요?"
1,매매결정,유사문장,롯데 푸드 주식 살까요?
2,매매결정,유사문장,미코 매도해야 될까요??
3,매매결정,유사문장,인터플렉스 매도할지 고민되네요
4,매매결정,유사문장,파세코 지금 종가로 사도 될까요??
5,매매결정,유사문장,후성 물렸는데 팔아야할까요
6,종목분석,대표문장,현대공업 종목 분석 부탁해
7,종목분석,유사문장,셀바스ai 어떤가요?


In [0]:
test_data.head(8)

,tag,sentence
0,거래,3시30분 장후 매수는 못하는건가여? 하려면 어떻게 해야하는건지??
1,거래,매도하고 며칠 뒤에 현금이 들어올 수 있죠?
2,거래,매도할 때 세금을 내는 건가요 매수할 때 세금을 내는 건가요
3,거래,미국 주식을 사고팔면 세금을 안 내도 되나요?
4,거래,비상장 주식을 살 수 있는 방법이 뭘까요?
5,거래,시간외 거래는 어떻게 하는건가요? 장단점이 있나요?
6,거래,시간외 거래량을 알고 싶은데 무얼 봐야하죠?
7,거래,오늘 팔면 오늘 인출이 불가능한가요?


In [0]:
# transform_text created by BIGTREE

def transform_text(data):
    ds = gluon.data.SimpleDataset([[data]])
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocabulary, lower=False) #vocabulary = gex_mxnet_kobert_model
    trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=64, pair =False)
    res = list(ds.transform(trans))

    input_ids =[]
    input_mask =[]
    token_type_ids=[]

    for data in list(res[0][0]):
        if data ==1 : # which is stopping at padding token (2: CLS, 3:SEP) # 이렇게 쓸 경우 문장의 길이가 모두 달라짐. 옳은 것인가?
            break
        input_ids.append(data)
        input_mask.append(1) # not masking at all
        token_type_ids.append(0) # only one sentence. No NSP.

    input_ids = torch.LongTensor([input_ids])
    input_mask = torch.LongTensor([input_mask])
    token_type_ids = torch.LongTensor([token_type_ids])

    return input_ids, input_mask, token_type_ids 

In [0]:
# Before torch.load, you have to load BERTClassifier.
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 config,
                 num_labels=20,
                 dropout=None,
                 ):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.num_labels = num_labels 
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size,num_labels)
       
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits

In [0]:
model = torch.load('./mykobert.pt')
model.eval()

In [0]:
def sent2BERT(sent):
    input_ids, input_mask, token_type_ids = transform_text(sent)
    _, pooled_output = model.bert(input_ids)
    vector = np.reshape(pooled_output[0].detach().numpy(),[1,-1])
    
    return vector[0]

In [0]:
exp_data['bertvec'] = [sent2BERT(x) for x in exp_data['sentence']]
test_data['bertvec'] = [sent2BERT(x) for x in test_data['sentence']]

In [0]:
# 인덱싱
exp_data=exp_data.set_index(['tag','type','sentence'])
test_data=test_data.set_index(['tag','sentence'])
exp_data.head()

bertvec
tag  type sentence                                                                          
매매결정 대표문장 LG유플러스 14,500원에 보유중인데 매도해야 할까요?  [0.65814394, -0.64247227, -0.54271704, -0.7552...
     유사문장 롯데 푸드 주식 살까요?                    [0.6874154, -0.6162413, -0.46924344, -0.741565...
          미코 매도해야 될까요??                    [0.6682263, -0.54477525, -0.57169276, -0.71691...
          인터플렉스 매도할지 고민되네요                 [0.5876868, -0.58916193, -0.43116197, -0.75308...
          파세코 지금 종가로 사도 될까요??              [0.60122687, -0.5770306, -0.19647239, -0.77037...

In [0]:
test_data.head()

bertvec
tag sentence                                                                                
거래  3시30분 장후 매수는 못하는건가여? 하려면 어떻게 해야하는건지??  [-0.5679633, 0.4752599, 0.43517652, -0.570345,...
    매도하고 며칠 뒤에 현금이 들어올 수 있죠?               [-0.719123, 0.5784102, 0.53249556, -0.5298613,...
    매도할 때 세금을 내는 건가요 매수할 때 세금을 내는 건가요      [-0.6540911, 0.50219375, 0.37031442, -0.545244...
    미국 주식을 사고팔면 세금을 안 내도 되나요?              [-0.6621398, 0.5330373, 0.35644805, -0.5292124...
    비상장 주식을 살 수 있는 방법이 뭘까요?                [-0.68325263, 0.5741373, 0.4419333, -0.4505839...

In [0]:
test_data.index

MultiIndex([('거래', ...),
            ('거래', ...),
            ('거래', ...),
            ('거래', ...),
            ('거래', ...),
            ('거래', ...),
            ('거래', ...),
            ('거래', ...),
            ('거래', ...),
            ('거래', ...),
            ...
            ('투자', ...),
            ('투자', ...),
            ('투자', ...),
            ('투자', ...),
            ('투자', ...),
            ('투자', ...),
            ('투자', ...),
            ('투자', ...),
            ('투자', ...),
            ('투자', ...)],
           names=['tag', 'sentence'], length=210)

In [0]:
cosine_similarity(exp_data['bertvec'][0].reshape(1,-1),test_data['bertvec'][0].reshape(1,-1))

array([[-0.08543762]], dtype=float32)

In [0]:
for test in test_data['bertvec']:
    tmp = [cosine_similarity(exp.reshape(1,-1),test.reshape(1,-1))for exp in exp_data['bertvec']]
    tmp2 = [tm[0][0] for tm in tmp]
    
    exp_data[str(test)]=tmp2

In [0]:
len(test_data)

210

In [0]:
len(exp_data.columns)

211

In [0]:
del exp_data['bertvec']

In [0]:
exp_data.head()

,,,...


In [0]:
# 인덱스 수정해주기
exp_data.columns=test_data.index

In [0]:
exp_data.head()

tag                                                                          거래  \
sentence                                  3시30분 장후 매수는 못하는건가여? 하려면 어떻게 해야하는건지??   
tag  type sentence                                                                
매매결정 대표문장 LG유플러스 14,500원에 보유중인데 매도해야 할까요?                             -0.085438   
     유사문장 롯데 푸드 주식 살까요?                                               -0.140080   
          미코 매도해야 될까요??                                               -0.168931   
          인터플렉스 매도할지 고민되네요                                            -0.084255   
          파세코 지금 종가로 사도 될까요??                                         -0.007994   

tag                                                                 \
sentence                                  매도하고 며칠 뒤에 현금이 들어올 수 있죠?   
tag  type sentence                                                   
매매결정 대표문장 LG유플러스 14,500원에 보유중인데 매도해야 할까요?                -0.114746   
     유사문장 롯데 푸드 주식 살까요?                                  -0.152842   
          미코 매도해야 될까요??                                  -0.173767   
          인터플렉스 매도할지 고민되네요                               -0.099943   
          파세코 지금 종가로 사도 될까요??                            -0.015558   

tag                                                                          \
sentence                                  매도할 때 세금을 내는 건가요 매수할 때 세금을 내는 건가요   
tag  type sentence                                                            
매매결정 대표문장 LG유플러스 14,500원에 보유중인데 매도해야 할까요?                         -0.133380   
     유사문장 롯데 푸드 주식 살까요?                                           -0.186538   
          미코 매도해야 될까요??                                           -0.212794   
          인터플렉스 매도할지 고민되네요                                        -0.122639   
          파세코 지금 종가로 사도 될까요??                                     -0.085511   

tag                                                                  \
sentence                                  미국 주식을 사고팔면 세금을 안 내도 되나요?   
tag  type sentence                                                    
매매결정 대표문장 LG유플러스 14,500원에 보유중인데 매도해야 할까요?                 -0.144246   
     유사문장 롯데 푸드 주식 살까요?                                   -0.201323   
          미코 매도해야 될까요??                                   -0.217536   
          인터플렉스 매도할지 고민되네요                                -0.136990   
          파세코 지금 종가로 사도 될까요??                             -0.103302   

tag                                                                \
sentence                                  비상장 주식을 살 수 있는 방법이 뭘까요?   
tag  type sentence                                                  
매매결정 대표문장 LG유플러스 14,500원에 보유중인데 매도해야 할까요?               -0.221948   
     유사문장 롯데 푸드 주식 살까요?                                 -0.283639   
          미코 매도해야 될까요??                                 -0.290281   
          인터플렉스 매도할지 고민되네요                              -0.216827   
          파세코 지금 종가로 사도 될까요??                           -0.174072   

tag                                                                     \
sentence                                  시간외 거래는 어떻게 하는건가요? 장단점이 있나요?   
tag  type sentence                                                       
매매결정 대표문장 LG유플러스 14,500원에 보유중인데 매도해야 할까요?                    -0.383812   
     유사문장 롯데 푸드 주식 살까요?                                      -0.455376   
          미코 매도해야 될까요??                                      -0.430991   
          인터플렉스 매도할지 고민되네요                                   -0.391405   
          파세코 지금 종가로 사도 될까요??                                -0.323301   

tag                                                                 \
sentence                                  시간외 거래량을 알고 싶은데 무얼 봐야하죠?   
tag  type sentence                                                   
매매결정 대표문장 LG유플러스 14,500원에 보유중인데 매도해야 할까요?                -0.337026   
     유사문장 롯데 푸드 주식 살까요?                                  -0.390808   
          미코 매도해야 될까요??                                  -0.416741   
      

In [0]:
exp_data.to_csv("bert_test_data.csv", encoding="cp949")

In [0]:
len(exp_data)

114